In [ ]:
pip install ipykernel

In [ ]:
pip install opencv-python numpy ultralytics sort

In [ ]:
import cv2
import numpy as np
from sort import Sort
from ultralytics import YOLO
import os

# 1. CONFIG
VIDEO_PATH   = 'C:/test_video/4.mp4'
LINE1_Y      = 400
LINE2_Y      = 500
REAL_DIST_M  = 10  # 

# 2. LOAD VIDEO & MODEL
cap     = cv2.VideoCapture(VIDEO_PATH)
model   = YOLO('yolov8n.pt')
tracker = Sort()

# 3. DATA STORAGE
entry_time_ms = {}
done_ids = set()

# 4. GET VIDEO INFO
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 5. DEFINE OUTPUT VIDEO PATH
output_path = os.path.splitext(VIDEO_PATH)[0] + '_output.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 6. PROCESS VIDEO
while True:
    ret, frame = cap.read()
    if not ret:
        break

    cur_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
    results = model.predict(frame, classes=[1,2,3,4,5,6], verbose=False)

    bboxes = results[0].boxes.xyxy.cpu().numpy()
    confs  = results[0].boxes.conf.cpu().numpy().reshape(-1, 1)
    dets   = np.hstack((bboxes, confs))
    tracks = tracker.update(dets)

    # Draw lines
    cv2.line(frame, (300, LINE1_Y), (width, LINE1_Y), (0, 255, 0), 2)
    cv2.line(frame, (300, LINE2_Y), (width, LINE2_Y), (0, 0, 255), 2)

    # Process each track
    for x1, y1, x2, y2, tid in tracks.astype(int):
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if cy >= LINE1_Y and tid not in entry_time_ms:
            entry_time_ms[tid] = cur_ms

        if tid in entry_time_ms and cy >= LINE2_Y and tid not in done_ids:
            elapsed_ms = cur_ms - entry_time_ms[tid]
            elapsed_s  = elapsed_ms / 1000.0

            if elapsed_s > 0:
                speed_m_s = REAL_DIST_M / elapsed_s
                speed_kmh = speed_m_s * 3.6

                done_ids.add(tid)

                cv2.putText(frame,
                            f"{speed_kmh:.1f} km/h",
                            (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.8, (255, 255, 0), 2)

        # Draw box and ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 0), 2)
        cv2.putText(frame, str(tid), (x1, y2 + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)

    # Show and save frame
    cv2.imshow('Speed Detection', frame)
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 7. RELEASE RESOURCES
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"[✅] Output saved to: {output_path}")